# Analiza korespondencji

In [ ]:
import os
os.getcwd() # oczekiwany .../AWXXXX/materialy/zajecia05
# mozna uzyc os.chdir("path") do zmiany

In [ ]:
import yaml
spec =  yaml.safe_load(open('../../spec.yaml'))

In [ ]:
# STATA
import stata_setup
stata_setup.config(spec["stata_path"], spec["stata_type"])
from pystata import stata

In [ ]:
import pandas as pd
import numpy as np
import scipy
import sklearn
import scipy.stats as stats
from sklearn import datasets

from CAwne.CA import CA, MCA

import pyAesCrypt

### Dekompozycja Macierzy

In [ ]:
# https://en.wikipedia.org/wiki/Matrix_decomposition

1. Przy rozwiązywaniu układu równań liniowych Ax=b
2. eigendecomposition czy rozkład według wartości osobliwych macierzy A

### SVD

![](https://upload.wikimedia.org/wikipedia/commons/thumb/b/bb/Singular-Value-Decomposition.svg/1920px-Singular-Value-Decomposition.svg.png)

In [ ]:
iris_set = datasets.load_iris()
# CENTRING
iris = iris_set["data"] - np.mean(iris_set["data"], axis = 0)

np.mean(iris, axis = 0),\
np.std(iris, axis = 0)

In [ ]:
u, s, vh = np.linalg.svd(iris, full_matrices = False)

In [ ]:
# SVD - X = U * S * Vt
np.allclose(iris, u @ np.diag(s) @ vh)

In [ ]:
# whiten - X_whiten = U * Vt
np.mean(u @ vh, axis = 0),\
np.std(u @ vh, axis = 0)

In [ ]:
# principal components - X_new = X * V = U * S * Vt * V = U * S 
np.allclose(u @ np.diag(s), iris @ vh.T)

In [ ]:
pd.DataFrame(iris @ vh.T).corr()

ZADANIE 1

Stan cywilny a kraj pochodzenia samochodu (przyklad zaczerpniety z http://v8doc.sas.com/sashtml/stat/chap24/sect26.htm)
Wykorzystujemy zbior danych, zawierajacy informacje o pochodzeniu samochodu (origin: USA, JAPONIA, EUROPA) 
i stanie cywilnym jego wlasiciela (marital: 1 'Wolny z dziecmi' 2 'Malzenstwo z dziecmi' 3 'Wolny' 4 'Malzenstwo'

Wczytanie danych

In [ ]:
if not os.path.isfile("../../dane/samochody.dta"):
    password = spec["password_pyaescrypt"]
    if password is None:
        password = input("password: ")
    pyAesCrypt.decryptFile("../../dane/samochody.dta.aes", "../../dane/samochody.dta", password)

In [ ]:
%%stata
use ../../dane/samochody.dta, clear

In [ ]:
samochody = pd.read_stata("../../dane/samochody.dta")

In [ ]:
# pd.crosstab(np.random.choice(["a","b","c", "d"], 100, True),np.random.choice([1,2,3], 100, True))

-------------------------

### Tabela Wielodzielna - cross table

lrchi2 - Likelihood Ratio Chi-Square - test ilorazu wiarygodnosci, opiera sie na sumowaniu po wszystkich komorkach wyrazenia postaci: 
Nij*ln(Nij/Eij), 
gdzie Nij - liczebnosci zaobserwowane, Eij - liczebnosci oczekiwane

nie ma podstaw do odrzucenia hipotezy zerowej o niezaleznosci -- nie powinno sie stosowac analizy korespondencji!

In [ ]:
%%stata
//2. Czy miedzy zmiennymi wystepuje zaleznosc?
tab origin marital, chi2 lrchi2

In [ ]:
pd.crosstab(samochody.origin, samochody.marital, margins = True)

In [ ]:
cross_sam = pd.crosstab(samochody.origin, samochody.marital)
stats.chi2_contingency(cross_sam)

In [ ]:
%%stata
tab origin marital, V

//               Cramér's V =   0.1110
// pomiedzy zmiennymi wystepuje bardzo slaby zwiazek (wartosci wspolczynnika V-Cramera <0.30)

In [ ]:
X2 = stats.chi2_contingency(cross_sam.values, correction=False)[0]
n = np.sum(cross_sam.values) # sample size
minDim = np.min(cross_sam.shape) - 1 # minimum of rows and columns
V = np.sqrt((X2/n) / minDim)
print(V)

In [ ]:
pd.crosstab(samochody.marital, samochody.origin, margins = False)

In [ ]:
pmat = pd.crosstab(samochody.marital, samochody.origin, margins = False)

### Analiza korespondencji


In [ ]:
%%stata
// uzywamy kiedy dane wejsciowe sa  w postaci standardowego arkusza danych - w wierszach obserwacje a w kolumnach  kolejne zmienne.*/ 
ca marital origin


/*1. "Inertia and Chi-Square Decomposition"
Singular value - wartosci wlasne uzyskane podczas dekompozycji SVD
Principal Inertia - wartosci wlasne podniesione do kwadratu - czesc inercji ("wariancji")
zwiazana z danym wymiarem
Chi-Square - "Inercja"*"liczba obserwacji" - czesc statystyki chi2 zwiazana z danym wymiarem
Percent - inercja danego wymiaru podzielona przez calkowita inercje - procent inercji ("zmiennosci") wyjasniony przez dany wymiar
Cumulative Percent - skumulowana poprzednia kolumna*/

/*2. "Row Coordinates"
Wspolrzedne odpowiadajace wierszom w przestrzeni dwuwymiarowej, czyli sa to kolumny 
macierzy U uzyskanych w wyniku dekompozycji SVD*/

/*3. "Summary Statistics for the Row Points"
QUALITY - miara ta przyjmuje wartosci z przedzialu [0,1] i mowi nam o jakosci 
reprezentacji poszczegolnych wierszy (kolumn) w przestrzeni o mniejszej liczbie wymiarow
Im blizej jednosci sa jej wartosci, tym lepiej reprezentowany jest analizowany punkt
w danej przestrzeni. U nas oczywiscie wartosc 1, bo tablica wymiaru 3x4 jest dokladnie 
reprezentowana w przestrzeni 2 wymiarowej (2 = min(3-2,4-1)).
MASS - Masa wiersza (lub kolumny) - mozna interpretowac jako range waznosci 
poszczegolnych wieszy. 
INERTIA - wzgledna bezwladnosc punktu, okresla udzial danego punktu w ogolnej inercji 
(bezwladnosci). Np. 'Malzenstwo z dziecmi' i 'Wolny' maja najwiekszy udzial w 
bezwladnosci (czyli zmiennosci) wsrod wierszy*/

/*4. "Partial Contributions to Inertia for the Row Points"
Rozklad ogolnej bezwladnosci na poszczegolne wymiary (osie). Jest to udzial punktu
w bezwladnosci wymiaru. Interpretujemy jako czesc bezwladnosci zwiazanej z konkretnym 
wymiarem, ktora jest wyjasniona przez dany punkt. Punkty z duzymi wartosciami udzialu 
w bezwladnosci w relatywnie duzym stopniu przyczynily sie do definicji danego wymiaru.
Sposob obliczenia: (masa wiersza) * (wspolrzedna wiersza w danym wymiarze)^2 / (wartosc
wlasna podniesiona do kwadratu dla danego wymiaru, czyli inercja danego wymiaru). 
Np. 'Malzenstwo' i I wymiar:
0.301 *(0.072)^2 / 0.02287 = 0.01016824. 
Interpretacja: 'Malzenstwo z dziecmi' i 'Wolny' w najwiekszym stopniu przyczynily sie do
zdefiniowania pierwszego wymiaru*/

/*5. Squared Cosines for the Row Points
Udzial wymiaru w bezwladnosci punktu. Definiujemy jako:
(wspolrzedna punktu w danym wymiarze)^2 / (kwadrat odleglosci punktu od srodka ukladu wspolrzednych
czyli przecietnego profila wierszowego lub kolumnowego). Jest to kwadrat cosinusa
kata miedzy odcinkiem laczacym srodek ukladu wspolrzednych z rozwazanym punktem a dana 
osia ukladu wspolrzednych. Obliczenia dla 'Malzenstwo' i I wymiaru: (-0.0278)^2 / ((-0.0278)^2 + (0.0134)^2) = 0.81
Interpretacja: I wymiar dobrze opisuje punkty reprezentujace 'Malzenstwo z dziecmi'
i 'Wolny'*/
//II wymiar dobrze opisuje punkt 'Wolny z dziecmi'

//W przypadku kolumn:
// I wymiar dobrze opisuje punkty 'USA' i 'Japonia'
// II wymiar dobrze opisuje  punkty 'USA' i 'Europa'


In [ ]:
%%stata
/*Przechodzimy do interpretacji wykresu 'Status rodzinny vs. pochodzenie samochodu'*/
cabiplot , norow yline(0) xline(0)
cabiplot , nocolumn yline(0) xline(0)
cabiplot, yline(0) xline(0)

/*Interpretacja: Zaczynamy od osobnej interpretacji punktow odpowiadajacych wierszom i kolumnom.
Wiersze:
EUROPA - wsporzedna dla wymiaru I bliska 0 - praktycznie brak wkladu w inercje I wymiaru; w ogole wklad w calkowita inercje jest nie wielki (bo wazniejszy jest I pierwszy wymiar),
duzy udzial w inercji II wymiaru (bo wspolrzedna dla drugiego wymiaru ma duza wartosc w porownaniu z pozostalymi punktami odpowiadajacymi wierszom). Polozenie tego punktu 
jest calkowicie determinowane przez II wymiar (potwierdza to kwadrat cosinusa). 
JAPONIA i USA - leza wzdluz I osi, maja duzy wklad do statystyki chi2 i do inercji I  wymiaru. Os pozioma rozroznia samochody Japonskie od Amerykanskich.
Kolumny:
'Malzenstwo' i 'Wolny z dziecmi' maja pierwsza wspolrzedna praktycznie rowna 0. Os pozioma rozroznia 'Wolnych' od 'Malzenstwo z dziecmi'
Interpetacja I osi wskazuje zatem na zaleznosc miedzy 'malzenstwo z dziecmi' a posiadanie samochodu produkcji amarykanskiej oraz byciu 'Wolnym' i posiadanie samochodu produkcji 
japonskiej. 

Podumowujac, analiza korespondencji wskazuje, iz osoby w zwiazku malzenskim posiadajace dzieci czesciej niz przy zalozeniu niezaleznosci miedzy wierszami 
a kolumnami tablicy kontyngencji kupuja samochody produkcji amerykanskiej, natomiast osoby stanu wolnego, bez dzieci czesciej niz przy zalozeniu niezaleznosci
miedzy wierszami a kolumnami tablicy kontyngencji uzywaja samochodow marki japonskiej.*/


/*Interpretacja: Zaczynamy od osobnej interpretacji punktow odpowiadajacych wierszom i kolumnom.
Wiersze:
EUROPA - wsporzedna dla wymiaru I bliska 0 - praktycznie brak wkladu w inercje I wymiaru; w ogole wklad w calkowita inercje jest nie wielki (bo wazniejszy jest I pierwszy wymiar),
duzy udzial w inercji II wymiaru (bo wspolrzedna dla drugiego wymiaru ma duza wartosc w porownaniu z pozostalymi punktami odpowiadajacymi wierszom). Polozenie tego punktu 
jest calkowicie determinowane przez II wymiar (potwierdza to kwadrat cosinusa). 
JAPONIA i USA - leza wzdluz I osi, maja duzy wklad do statystyki chi2 i do inercji I  wymiaru. Os pozioma rozroznia samochody Japonskie od Amerykanskich.
Kolumny:
'Malzenstwo' i 'Wolny z dziecmi' maja pierwsza wspolrzedna praktycznie rowna 0. Os pozioma rozroznia 'Wolnych' od 'Malzenstwo z dziecmi'
Interpetacja I osi wskazuje zatem na zaleznosc miedzy 'malzenstwo z dziecmi' a posiadanie samochodu produkcji amarykanskiej oraz byciu 'Wolnym' i posiadanie samochodu produkcji 
japonskiej. 

Podumowujac, analiza korespondencji wskazuje, iz osoby w zwiazku malzenskim posiadajace dzieci czesciej niz przy zalozeniu niezaleznosci miedzy wierszami 
a kolumnami tablicy kontyngencji kupuja samochody produkcji amerykanskiej, natomiast osoby stanu wolnego, bez dzieci czesciej niz przy zalozeniu niezaleznosci
miedzy wierszami a kolumnami tablicy kontyngencji uzywaja samochodow marki japonskiej.*/

------------------

In [ ]:
cc = CA(pmat)

In [ ]:
cc.inertia

In [ ]:
np.round(cc.eigenvals, 3)

In [ ]:
# F principal coordinates of rows
# G principal coordinates of cols
cc.F, cc.G

In [ ]:
from CAwne.CA import CA
cc.scree_diagram()

In [ ]:
cc.plot()

### ZADANIE 2

Kontrowersje -- czy wartosci Polakow zmienily sie pomiedzy 1989 a 2012 rokiem?

Baza do zadania zostala stworzona na podstawie badania World Values Survey (http://www.worldvaluessurvey.org/wvs.jsp [dostep 20 marca 2015]),
a dokladnie jego 4 fal, w ktorych brala udzial Polska:  
fali 2 : 1989 (938 osob)  
fali 3 : 1997 (1153 osoby)  
fali 5 : 2005 (1000 osob)  
fali 6 : 2012 (966)  
 
W kazdym badaniu respondentom zadano pytania, jak bardzo akceptowalne sa zjawiska z okreslonej puli.
Respondenci odpowiadali wybierajac jedna z kilku mozliwych odpowiedzi, porzadkujacych ich stopien akceptacji danego zjawiska
Dla potrzeb tego zadania policzono osoby, ktore na pytania udzielily odpowiedzi "calkowicie nieakceptowalne". Nastepnie przeliczono
jaki procent wszystkich odpowiedzi stanowi tak zdecydowanie okreslony brak akceptacji.

Uzyte zmienne:
kontrowersja: zjawisko, ktore wywoluje kontrowersje, gdzie	1 -- pobieranie niezasluzonych swiadczen od rzadu; 2 -- oszustwo podatkowe;
3 -- kupno skradzionych rzeczy; 4 -- lapowkarstwo; 5 -- homoseksualnosc; 6 -- prostytucja; 7 -- aborcja; 8 -- rozwod; 9 -- eutanazja; 10 -- samobojstwo

rok -- rok, w ktorym przeprowadzono wywiad, gdzie 1 -- 1989; 2 -- 1997; 3 -- 2005; 4 --2012;

wartosc -- procent respondentow, ktorzy na pytanie o okreslone zjawisko odpowiedzieli, ze jest "calkowicie nieakceptowalne"

0. Uzycie bazy danych

In [ ]:
%%stata
use ../../dane/kontrowersje.dta, clear

In [ ]:
kontrowersje = stata.pdataframe_from_data(valuelabel = True)

In [ ]:
kontrowersje["kontrowersja"] = kontrowersje.kontrowersja.map({1:"niezasluzone swiadczenia", 2: "oszustwo podatkowe", 3: "kupno skradzionych rzeczy", 4: "lapowkarstwo", 5: "homoseksualnosc", 6: "prostytucja", 7: "aborcja", 8: "rozwod", 9: "eutanazja", 10: "samobojstwo"}).astype("category")
kontrowersje["rok"] = kontrowersje.rok.map({1: "1989", 2: "1997", 3: "2005", 4:"2012"}).astype("category")

In [ ]:
%%stata
//1. Nadanie etykiet wartosciom zmiennych "kontrowersja" i "rok"
label define pytania 1 "niezasluzone swiadczenia" 2 "oszustwo podatkowe" 3 "kupno skradzionych rzeczy" 4 "lapowkarstwo" 5 "homoseksualnosc" 6 "prostytucja" 7 "aborcja" 8 "rozwod" 9 "eutanazja" 10 "samobojstwo"
label values kontrowersja pytania

label define rok 1 "1989" 2 "1997" 3 "2005" 4 "2012"
label values rok rok

In [ ]:
# %stata help weight

In [ ]:
%%stata
//2. Czy miedzy zmiennymi wystepuje zaleznosc?
tabulate rok kontrowersja [fw=wartosc], chi2 lrchi2 
tabulate rok kontrowersja [fw=wartosc], V

In [ ]:
kontrowersje.pivot_table(values="wartosc", index="rok", columns="kontrowersja")

In [ ]:
%%stata
//3. Analiza korespondencji
ca kontrowersja rok [fw=wartosc]

In [ ]:
%%stata
//4. Prezentacja graficzna wynikow
cabiplot , norow yline(0) xline(0)
cabiplot , nocolumn yline(0) xline(0)
cabiplot, yline(0) xline(0)

PYTHON

In [ ]:
ca = CA(kontrowersje.pivot_table(values="wartosc", index="rok", columns="kontrowersja"))

In [ ]:
ca.inertia

In [ ]:
np.round(ca.eigenvals, 3)

In [ ]:
ca.plot()

### MCA - Wielokrotna Analiza Korespondencji

In [ ]:
iris2 = pd.DataFrame({"sepal_length": pd.cut(iris[:, 0], 3, labels = ["small", "medium", "high"]),
                      "sepal_width": pd.cut(iris[:, 1], 3, labels = ["small", "medium", "high"]),
                      "name": pd.Series(iris_set["target"]).astype('category').cat.rename_categories(iris_set["target_names"].tolist())
                     })

In [ ]:
%stata clear

In [ ]:
stata.pdataframe_to_data(iris2)

In [ ]:
%%stata
encode sepal_length, gen(sepal_length_c)
encode sepal_width, gen(sepal_width_c)
encode name, gen(name_c)

In [ ]:
%%stata
mca sepal_length_c sepal_width_c name_c

In [ ]:
%%stata
mcaplot, overlay

In [ ]:
ca2 = MCA(pd.get_dummies(iris2))

In [ ]:
np.round(ca2.eigenvals, 3)

In [ ]:
ca2.plot()

### Zadanie 3

Wypadki przy pracy  -- czy istnieje zwiazek pomiedzy powodem wypadku a branza?

Baza do zadania zostala stworzona na podstawie danych udostepnionych przez GUS z ankiety Z11
http://stat.gov.pl/obszary-tematyczne/rynek-pracy/warunki-pracy-wypadki-przy-pracy/wypadki-przy-pracy-w-2013-r-,4,6.html [dostep dnia 15 marca 2015]
Wybrano 11 branz (branza) : 1 "rolnictwo" 2 "gornictwo" 3 "przetworstwo" 4 "profesjonalisci" 5 "budownictwo" 6 "handel" 7 "gastronomia" 8 "transport" 9 "finanse i ubezpieczenia" 10 "edukacja" 11 "zdrowie"
oraz 9 powodow (powod): 1 "niewlasciwy stan czynnika materialnego" 2 "niewlasciwa organizacja pracy" 3 "niewlasciwa organizacja stanowiska pracy" 4 "niewlasciwe poslugiwanie sie czynnikiem materialnym" 5 "brak sprzetu ochronnego" 6 "samowola pracownika" 7 "zly stan psychofizyczny pracownika" 8 "nieprawidlowe zachowanie pracownika" 9 "inne"

In [ ]:
%%stata
import delimited "../../dane/wypadki.csv", delimiter(";") clear

In [ ]:
%stata rename (v1 v2 v3) (branza powod liczba)

In [ ]:
wypadki = pd.read_csv("../../dane/wypadki.csv", sep = ";", header=0)
wypadki.columns = ["branza", "powod", "liczba"]

- 1. Nadaj etykiety wartosciom zmiennych "branza" i "powod"
- 2. Czy pomiedzy zmiennymi wystepuje zaleznosc? Zinterpretuj wyniki testu chi-kwadrat i wspolczynnik V-Cramera
- 3. Przeprowadz analize korespondencji. Jaki procent inercji tlumaczy rozwiazanie dwuwymiarowe?
- 4. Zinterpretuj uzyskane wyniki prezentacji graficznej.